In [1]:
## import libraries
import os, sys
import yaml
import re
import xarray as xr
import pandas as pd
import numpy as np
from datetime import timedelta
%matplotlib inline
from datetime import timedelta

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import textwrap

# import personal modules
sys.path.append('../modules')
import custom_cmaps as ccmap
from plotter import draw_basemap
import ar_funcs
# dask.config.set(**{'array.slicing.split_large_chunks': True})

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/SEAK-impacts/share/proj failed


In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
def compare_mclimate_to_forecast(fc, mclimate):
    ## compare IVT forecast to mclimate
    b_lst = []
    quant_lst = [0.  , 0.75, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.  ]
    nquantiles = len(quant_lst)
    for i, quant in enumerate(range(nquantiles)):
        bottom = mclimate.ivt.isel(quantile=quant).squeeze() # minimum threshold
        
        if i == 0:
            # only need to see where IVT in the forecast is less than minimum quantile
            b = xr.where(cond=fc.IVT < bottom, x=quant_lst[i], y=np.nan)
            
        elif (i > 0) & (i < nquantiles-1):
            # where IVT in the forecast is greater than current quartile, but less than next quartile
            top = mclimate.ivt.isel(quantile=i+1)
            b = xr.where(cond=(fc.IVT > bottom) & (fc.IVT < top), x=quant_lst[i], y=np.nan)
            
        elif (i == nquantiles-1):
            # where IVT is greater than final quartile
            b = xr.where(cond=(fc.IVT > bottom), x=quant_lst[i], y=np.nan)
    
        b.name = 'ivt'
        
        var_dict = {'ivt_mclimate': (['lat', 'lon'], b.squeeze().values)}
        new_ds = xr.Dataset(var_dict,
                        coords={'lat': (['lat'], b.lat.values),
                                'lon': (['lon'], b.lon.values)})
        
        b_lst.append(new_ds)
            
    ds = xr.merge(b_lst)

    return ds

def load_reforecast(date, varname):
    fname = path_to_data + 'preprocessed/GEFSv12_reforecast/{0}/{1}_{0}_F51_F72.nc'.format(varname, date)
    forecast = xr.open_dataset(fname)
    forecast = forecast.rename({'longitude': 'lon', 'latitude': 'lat', 'ivt': 'IVT'}) # need to rename this to match GEFSv12 Reforecast
    forecast = forecast.sel(lon=slice(-179.5, -110.))
    forecast = forecast.drop_vars(["ivtu", "ivtv"])
    forecast = forecast.isel(step=-1) ### need to fix this so it selects the right time step based on input
    forecast = forecast.mean('number') # ensemble mean

    return forecast

def load_mclimate(mon, day, lead=72):
    ## load mclimate data
    fname_pattern = path_to_data + 'preprocessed/mclimate/GEFSv12_reforecast_mclimate_ivt_{0}{1}_*hr-lead.nc'.format(mon, day)
    # print(fname_pattern)
    ds = xr.open_mfdataset(fname_pattern, engine='netcdf4', concat_dim="step", combine='nested')
    ds = ds.sortby("step") # sort by step (forecast lead)
    ds = ds.rename({'longitude': 'lon', 'latitude': 'lat'}) # need to rename this to match GEFSv12 Reforecast
    ds = ds.sel(step=lead, lon=slice(-179.5, -110.), lat=slice(70., 10.))

    return ds


In [4]:
# get list of all AR days
fname = '../out/SEAK_ardates_daily.csv'
ar_df = pd.read_csv(fname) # read in AR dates
idx = (ar_df.AR == 1)
ar_df = ar_df.loc[idx]
# reset the index as "time"
ar_df = ar_df.set_index(pd.to_datetime(ar_df['Unnamed: 0']))
## subset dates to Jan 2000 - Dec 2019
idx = (ar_df.index.year >= 2000) & (ar_df.index.year <= 2019)
ar_df = ar_df.loc[idx]
ar_dates = ar_df.index.values
ar_dates

array(['2000-01-06T00:00:00.000000000', '2000-01-08T00:00:00.000000000',
       '2000-01-19T00:00:00.000000000', ...,
       '2019-08-22T00:00:00.000000000', '2019-08-23T00:00:00.000000000',
       '2019-08-26T00:00:00.000000000'], dtype='datetime64[ns]')

In [5]:
fname = path_to_data + 'preprocessed/mclimate_AR_dates/mclimate_ivt_20000103_F72.nc'
ds = xr.open_dataset(fname)
ds


<xarray.Dataset>
Dimensions:       (lat: 241, lon: 279)
Coordinates:
  * lat           (lat) float64 70.0 69.75 69.5 69.25 ... 10.75 10.5 10.25 10.0
  * lon           (lon) float64 -179.5 -179.2 -179.0 ... -110.5 -110.2 -110.0
    time          datetime64[ns] ...
Data variables:
    ivt_mclimate  (lat, lon) float64 ...

In [9]:

dates_new = []
mon_lst = []
day_lst = []
for i, date in enumerate(ar_dates):
    # ts = pd.to_datetime(str(date))
    ts = date
    d = ts - timedelta(days=3)
    t = d.strftime('%Y%m%d')
    mon_lst.append(d.month)
    day_lst.append(d.day)
    dates_new.append(t)

## now compare mclimate to each forecast
ds_lst = []
for j, dates in enumerate(dates_new):
    try:
        fc = load_reforecast(dates, 'ivt')
        mclimate = load_mclimate(mon_lst[j], day_lst[j])
        
        ds = compare_mclimate_to_forecast(fc, mclimate)
        ds_lst.append(ds)
    except OSError:
        pass

ds_final = xr.concat(ds_lst, dim="impact_dates")
ds_final

KeyboardInterrupt: 

In [ ]:
## save data to netCDF file
print('Writing to netCDF ....')
out_fname = path_to_data + 'preprocessed/mclimate_ivt_ar_days_F72.nc'
ds_final.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')